# Analysis of multigrid restriction and prolongation
1D Poisson problem, discretized with a second order finite difference formula, yields the following linear system

In [22]:
import sympy as sym
sym.init_printing()


h = sym.symbols('h')
n = 9
m = 1 + (n-1)//2


def build_poisson_operator(n):
    A = sym.zeros(n,n)
    
    A[0,0] = 1
    A[n-1,n-1] = 1
    
    for i in range(1,n-1):
        A[i,i-1] = -1 / h**2
        A[i,i]   =  2 / h**2
        A[i,i+1] = -1 / h**2
        
    return A

A = build_poisson_operator(n)
A

⎡ 1    0    0    0    0    0    0    0    0 ⎤
⎢                                           ⎥
⎢-1   2    -1                               ⎥
⎢───  ──   ───   0    0    0    0    0    0 ⎥
⎢  2   2     2                              ⎥
⎢ h   h     h                               ⎥
⎢                                           ⎥
⎢     -1   2    -1                          ⎥
⎢ 0   ───  ──   ───   0    0    0    0    0 ⎥
⎢       2   2     2                         ⎥
⎢      h   h     h                          ⎥
⎢                                           ⎥
⎢          -1   2    -1                     ⎥
⎢ 0    0   ───  ──   ───   0    0    0    0 ⎥
⎢            2   2     2                    ⎥
⎢           h   h     h                     ⎥
⎢                                           ⎥
⎢               -1   2    -1                ⎥
⎢ 0    0    0   ───  ──   ───   0    0    0 ⎥
⎢                 2   2     2               ⎥
⎢                h   h     h                ⎥
⎢                                 

The projection on a coarser grid is supposed to work on smooth error components. As for "A multigrid tutorial" the operator for relaxing on the coarser grid is:

$$ A^{2h} := I_{h}^{2h}A^{h}I_{2h}^{h} $$

There is freedom in determining the restriction and prolongation operators, but it is advisable that

$$ I_{h}^{2h} = c(I_{2h}^{h})^{T} $$

A careful choice results in $A^{2h}$ being the matrix of the original problem, discretized on a grid double the size. That is desirable because it simplifies the solver



## Naive choices

In [23]:
def injective_restriction(n):
    m = 1 + (n-1)//2
    R = sym.zeros(m,n)
    
    for i in range(m):
        R[i,2*i] = 1
        
    return R

injective_restriction(n)

⎡1  0  0  0  0  0  0  0  0⎤
⎢                         ⎥
⎢0  0  1  0  0  0  0  0  0⎥
⎢                         ⎥
⎢0  0  0  0  1  0  0  0  0⎥
⎢                         ⎥
⎢0  0  0  0  0  0  1  0  0⎥
⎢                         ⎥
⎣0  0  0  0  0  0  0  0  1⎦

In [3]:
simple_projection(n)

⎡1  0  0  0  0⎤
⎢             ⎥
⎢0  0  1  0  0⎥
⎢             ⎥
⎣0  0  0  0  1⎦

In [26]:
def linear_prolongation(m):
    n = 2 * (m - 1) + 1
    P = sym.zeros(n,m)
    
    for i in range(n):
        if i % 2 == 0:
            P[i,i//2] = 1
        else:
            P[i,i//2]     = sym.nsimplify(0.5)
            P[i,(i+1)//2] = sym.nsimplify(0.5)
            
    return P

linear_prolongation(m)

⎡ 1    0    0    0    0 ⎤
⎢                       ⎥
⎢1/2  1/2   0    0    0 ⎥
⎢                       ⎥
⎢ 0    1    0    0    0 ⎥
⎢                       ⎥
⎢ 0   1/2  1/2   0    0 ⎥
⎢                       ⎥
⎢ 0    0    1    0    0 ⎥
⎢                       ⎥
⎢ 0    0   1/2  1/2   0 ⎥
⎢                       ⎥
⎢ 0    0    0    1    0 ⎥
⎢                       ⎥
⎢ 0    0    0   1/2  1/2⎥
⎢                       ⎥
⎣ 0    0    0    0    1 ⎦

In [31]:
injective_restriction(n) @ A @ linear_prolongation(m)

⎡ 1     0     0     0     0  ⎤
⎢                            ⎥
⎢-1     1    -1              ⎥
⎢────   ──   ────   0     0  ⎥
⎢   2    2      2            ⎥
⎢2⋅h    h    2⋅h             ⎥
⎢                            ⎥
⎢      -1     1    -1        ⎥
⎢ 0    ────   ──   ────   0  ⎥
⎢         2    2      2      ⎥
⎢      2⋅h    h    2⋅h       ⎥
⎢                            ⎥
⎢            -1     1    -1  ⎥
⎢ 0     0    ────   ──   ────⎥
⎢               2    2      2⎥
⎢            2⋅h    h    2⋅h ⎥
⎢                            ⎥
⎣ 0     0     0     0     1  ⎦

In [32]:
linear_prolongation(m).T @ A @ linear_prolongation(m)

⎡ 1     0     0     0     0  ⎤
⎢                            ⎥
⎢-1     1    -1              ⎥
⎢────   ──   ────   0     0  ⎥
⎢   2    2      2            ⎥
⎢2⋅h    h    2⋅h             ⎥
⎢                            ⎥
⎢      -1     1    -1        ⎥
⎢ 0    ────   ──   ────   0  ⎥
⎢         2    2      2      ⎥
⎢      2⋅h    h    2⋅h       ⎥
⎢                            ⎥
⎢            -1     1    -1  ⎥
⎢ 0     0    ────   ──   ────⎥
⎢               2    2      2⎥
⎢            2⋅h    h    2⋅h ⎥
⎢                            ⎥
⎣ 0     0     0     0     1  ⎦

## Full weight restriction

In [66]:
def full_weight_restriction(n):
    m = 1 + (n-1)//2
    
    R = sym.zeros(m,n)
    R[0,0]   = 1
    R[-1,-1] = 1
    
    for i in range(1,m-1):
        R[i,2*i-1] = sym.nsimplify(0.25)
        R[i,2*i]   = sym.nsimplify(0.5)
        R[i,2*i+1] = sym.nsimplify(0.25)
    
    return R

full_weight_restriction(n)

⎡1   0    0    0    0    0    0    0   0⎤
⎢                                       ⎥
⎢0  1/4  1/2  1/4   0    0    0    0   0⎥
⎢                                       ⎥
⎢0   0    0   1/4  1/2  1/4   0    0   0⎥
⎢                                       ⎥
⎢0   0    0    0    0   1/4  1/2  1/4  0⎥
⎢                                       ⎥
⎣0   0    0    0    0    0    0    0   1⎦

In [68]:
def full_weight_prolongation(m):
    n = 1 + 2*(m-1)
    
    return 2 * full_weight_restriction(n).T

full_weight_prolongation(m)

⎡2   0    0    0   0⎤
⎢                   ⎥
⎢0  1/2   0    0   0⎥
⎢                   ⎥
⎢0   1    0    0   0⎥
⎢                   ⎥
⎢0  1/2  1/2   0   0⎥
⎢                   ⎥
⎢0   0    1    0   0⎥
⎢                   ⎥
⎢0   0   1/2  1/2  0⎥
⎢                   ⎥
⎢0   0    0    1   0⎥
⎢                   ⎥
⎢0   0    0   1/2  0⎥
⎢                   ⎥
⎣0   0    0    0   2⎦

In [70]:
full_weight_restriction(n) @ A @ full_weight_prolongation(m)

⎡ 2     0     0     0     0  ⎤
⎢                            ⎥
⎢-1     1    -1              ⎥
⎢────  ────  ────   0     0  ⎥
⎢   2     2     2            ⎥
⎢2⋅h   2⋅h   4⋅h             ⎥
⎢                            ⎥
⎢      -1     1    -1        ⎥
⎢ 0    ────  ────  ────   0  ⎥
⎢         2     2     2      ⎥
⎢      4⋅h   2⋅h   4⋅h       ⎥
⎢                            ⎥
⎢            -1     1    -1  ⎥
⎢ 0     0    ────  ────  ────⎥
⎢               2     2     2⎥
⎢            4⋅h   2⋅h   2⋅h ⎥
⎢                            ⎥
⎣ 0     0     0     0     2  ⎦

That corresponds to the stencil of the discretization with spatial step $2h$, except for the boundary terms. That is still ok because the error and residuals at the boundary are always zero.